In [1]:
# SISTEMA DE RECOMENDACIÓN CORREGIDO
# Compatible con el modelo sin overfitting (sin country/quality_rating)

import pandas as pd
import numpy as np
import joblib
from datetime import datetime

print("SISTEMA DE RECOMENDACIÓN DE PROVEEDORES - VERSIÓN CORREGIDA")
print("Dewatering Solutions - Modelo sin Overfitting")
print("=" * 60)

# CARGAR MODELO Y DATOS CORREGIDOS
print("CARGANDO MODELO CORREGIDO...")

try:
    # Cargar modelo corregido
    model = joblib.load('../models/decision_tree_model_fixed.pkl')
    label_encoders = joblib.load('../models/label_encoders_fixed.pkl')
    target_encoder = joblib.load('../models/target_encoder_fixed.pkl')
    feature_names = joblib.load('../models/feature_names_fixed.pkl')
    
    print(f"   Modelo corregido cargado correctamente")
    print(f"   Features del modelo: {feature_names}")
    
except FileNotFoundError:
    print("   ERROR: Archivos del modelo corregido no encontrados")
    print("   Asegúrate de haber ejecutado el notebook 02b_fix_overfitting.ipynb")
    print("   Los archivos requeridos son:")
    print("   - ../models/decision_tree_model_fixed.pkl")
    print("   - ../models/label_encoders_fixed.pkl")
    print("   - ../models/target_encoder_fixed.pkl")
    print("   - ../models/feature_names_fixed.pkl")

# Cargar dataset original para análisis
df = pd.read_csv('../data/dewatering_realistic_supplier_dataset.csv')

print(f"   Proveedores disponibles: {len(target_encoder.classes_)}")

# Información de proveedores (SIN usar quality_rating como feature del modelo)
suppliers_info = df.groupby('supplier_name').agg({
    'quality_rating': 'first',  # Solo para mostrar info, no para predicción
    'country': 'first',         # Solo para mostrar info, no para predicción
    'delivery_days': 'first',
    'payment_terms_days': 'first',
    'price_usd': 'mean'
}).round(2)

print(f"\nINFORMACIÓN DE PROVEEDORES:")
for supplier, info in suppliers_info.iterrows():
    print(f"   {supplier}: Calidad {info['quality_rating']}/5.0, {info['country']}, {info['delivery_days']} días")

# FUNCIÓN DE RECOMENDACIÓN CORREGIDA
def recommend_suppliers(product_type, urgency, quantity, budget=10000):
    """
    Recomienda proveedores basado en los parámetros de entrada
    VERSIÓN CORREGIDA: Sin usar country/quality_rating como features del modelo
    
    Args:
        product_type (str): Tipo de producto requerido
        urgency (str): Nivel de urgencia ('Low', 'Medium', 'High', 'Critical')
        quantity (int): Cantidad requerida
        budget (float): Presupuesto disponible
    
    Returns:
        dict: Recomendaciones con scores y detalles
    """
    
    # Validar inputs
    valid_urgencies = ['Low', 'Medium', 'High', 'Critical']
    if urgency not in valid_urgencies:
        return {"error": f"Urgencia debe ser una de: {valid_urgencies}"}
    
    # Obtener productos disponibles del tipo solicitado
    available_products = df[df['product_type'] == product_type]
    if available_products.empty:
        available_types = df['product_type'].unique()
        return {"error": f"Producto no encontrado. Tipos disponibles: {list(available_types)}"}
    
    # Generar recomendaciones para cada proveedor que ofrece el producto
    recommendations = []
    
    for supplier_name in available_products['supplier_name'].unique():
        supplier_data = available_products[available_products['supplier_name'] == supplier_name].iloc[0]
        
        # FEATURES LIMPIAS (sin country/quality_rating)
        # Solo usamos las features que el modelo corregido acepta
        input_data = pd.DataFrame({
            'price_usd': [supplier_data['price_usd']],
            'delivery_days': [supplier_data['delivery_days']],
            'payment_terms_days': [supplier_data['payment_terms_days']],
            'shipping_included': [int(supplier_data['shipping_included'])],
            'express_available': [int(supplier_data['express_available'])],
            'order_urgency': [urgency],
            'quantity_needed': [quantity],
            'budget_available': [budget],
            'product_type': [product_type],
            'incoterms': [supplier_data['incoterms']],
            'month': [datetime.now().month],
            'quarter': [f"Q{(datetime.now().month-1)//3 + 1}"]
        })
        
        # Aplicar encoding (solo a las features que el modelo espera)
        for feature in label_encoders.keys():
            if feature in input_data.columns:
                try:
                    input_data[feature] = label_encoders[feature].transform(input_data[feature].astype(str))
                except ValueError:
                    # Si la categoría no existe, usar la primera categoría conocida
                    input_data[feature] = 0
        
        # Reordenar columnas según el modelo corregido
        input_data = input_data[feature_names]
        
        # Obtener probabilidades de predicción
        probabilities = model.predict_proba(input_data)[0]
        supplier_idx = np.where(target_encoder.classes_ == supplier_name)[0]
        
        if len(supplier_idx) > 0:
            probability = probabilities[supplier_idx[0]]
        else:
            probability = 0.0
        
        # Calcular costos
        total_cost = supplier_data['price_usd'] * quantity
        if not supplier_data['shipping_included']:
            total_cost += 300  # Costo estimado de envío
        
        # Crear recomendación con información completa (incluyendo quality_rating para mostrar)
        recommendation = {
            'supplier_name': supplier_name,
            'country': supplier_data['country'],                    # Para mostrar, no para ML
            'quality_rating': supplier_data['quality_rating'],      # Para mostrar, no para ML
            'price_usd': supplier_data['price_usd'],
            'total_cost': round(total_cost, 2),
            'delivery_days': supplier_data['delivery_days'],
            'payment_terms_days': supplier_data['payment_terms_days'],
            'probability_score': round(probability, 3),
            'within_budget': total_cost <= budget,
            'shipping_included': supplier_data['shipping_included'],
            'express_available': supplier_data['express_available']
        }
        
        # Calcular score final combinando:
        # 1. Probabilidad del modelo (40%)
        # 2. Factores de negocio externos (60%)
        
        # Score de calidad (externo al modelo)
        quality_score = recommendation['quality_rating'] / 5.0
        
        # Score de precio
        price_score = 1.0 if recommendation['within_budget'] else 0.5
        
        # Score de delivery
        delivery_score = 1.0 if recommendation['delivery_days'] <= 15 else 0.7
        
        # Score de términos de pago
        payment_score = 1.0 if recommendation['payment_terms_days'] > 0 else 0.8
        
        final_score = (
            recommendation['probability_score'] * 0.4 +  # Modelo ML
            quality_score * 0.25 +                       # Calidad (externo)
            price_score * 0.2 +                          # Presupuesto
            delivery_score * 0.1 +                       # Delivery
            payment_score * 0.05                         # Términos pago
        )
        
        recommendation['final_score'] = round(final_score, 3)
        
        # Nivel de recomendación
        if final_score >= 0.75:
            recommendation['recommendation_level'] = "Altamente Recomendado"
        elif final_score >= 0.6:
            recommendation['recommendation_level'] = "Recomendado"
        elif final_score >= 0.45:
            recommendation['recommendation_level'] = "Aceptable"
        else:
            recommendation['recommendation_level'] = "No Recomendado"
        
        recommendations.append(recommendation)
    
    # Ordenar por score final
    recommendations.sort(key=lambda x: x['final_score'], reverse=True)
    
    return {
        'product_type': product_type,
        'urgency': urgency,
        'quantity': quantity,
        'budget': budget,
        'recommendations': recommendations,
        'total_options': len(recommendations)
    }

# EJEMPLOS DE USO DEL SISTEMA CORREGIDO
print("\nEJEMPLOS DE USO DEL SISTEMA CORREGIDO")
print("=" * 40)

# Ejemplo 1: Filtros de prensa urgentes
print("\nEJEMPLO 1: FILTROS DE PRENSA URGENTES")
result1 = recommend_suppliers("filter_press_cloth_set", "High", 50, 2000)

if 'error' not in result1:
    print(f"Producto: {result1['product_type']}")
    print(f"Urgencia: {result1['urgency']}")
    print(f"Top 3 recomendaciones:")
    for i, supplier in enumerate(result1['recommendations'][:3], 1):
        print(f"{i}. {supplier['supplier_name']} ({supplier['country']}):")
        print(f"   Recomendación: {supplier['recommendation_level']}")
        print(f"   Score final: {supplier['final_score']}")
        print(f"   Calidad: {supplier['quality_rating']}/5.0")
        print(f"   Total: ${supplier['total_cost']}")
        print(f"   Delivery: {supplier['delivery_days']} días")
        print(f"   ML Score: {supplier['probability_score']}")
        print()
else:
    print(f"Error: {result1['error']}")

# Ejemplo 2: Sensores de presión para mantenimiento  
print("\nEJEMPLO 2: SENSORES DE PRESIÓN")
result2 = recommend_suppliers("pressure_sensor_digital", "Medium", 10, 1000)

if 'error' not in result2:
    print(f"Producto: {result2['product_type']}")
    print(f"Urgencia: {result2['urgency']}")
    print(f"Recomendaciones disponibles:")
    for i, supplier in enumerate(result2['recommendations'], 1):
        print(f"{i}. {supplier['supplier_name']} ({supplier['country']}):")
        print(f"   Recomendación: {supplier['recommendation_level']}")
        print(f"   Score final: {supplier['final_score']}")
        print(f"   Total: ${supplier['total_cost']}")
        print(f"   ML Score: {supplier['probability_score']}")
        print()
else:
    print(f"Error: {result2['error']}")

# Ejemplo 3: Rollos de tela filtrante gran volumen
print("\nEJEMPLO 3: ROLLOS DE TELA FILTRANTE")
result3 = recommend_suppliers("filter_cloth_roll", "Low", 2, 8000)

if 'error' not in result3:
    print(f"Producto: {result3['product_type']}")
    print(f"Presupuesto: ${result3['budget']}")
    print(f"Top recomendaciones:")
    for i, supplier in enumerate(result3['recommendations'], 1):
        print(f"{i}. {supplier['supplier_name']} ({supplier['country']}):")
        print(f"   Recomendación: {supplier['recommendation_level']}")
        print(f"   Score final: {supplier['final_score']}")
        print(f"   Calidad: {supplier['quality_rating']}/5.0")
        print(f"   Total: ${supplier['total_cost']}")
        print(f"   Dentro de presupuesto: {'Sí' if supplier['within_budget'] else 'No'}")
        print()
else:
    print(f"Error: {result3['error']}")

# FUNCIÓN SIMPLIFICADA PARA UI
def simple_search(product, urgency, quantity):
    """Versión simplificada para interfaces web"""
    result = recommend_suppliers(product, urgency, quantity)
    
    if 'error' in result:
        return result
    
    # Top 3 simplificado
    top_3 = []
    for supplier in result['recommendations'][:3]:
        top_3.append({
            'name': supplier['supplier_name'],
            'score': supplier['final_score'],
            'total_cost': supplier['total_cost'],
            'delivery': supplier['delivery_days'],
            'quality': supplier['quality_rating'],
            'recommendation': supplier['recommendation_level'],
            'country': supplier['country']
        })
    
    return {
        'product': product,
        'urgency': urgency,
        'quantity': quantity,
        'top_3': top_3
    }

# Probar función simplificada
print("\nEJEMPLO 4: BÚSQUEDA SIMPLIFICADA")
simple_result = simple_search("sensor_inductivo", "High", 5)

if 'error' not in simple_result:
    print(f"Producto: {simple_result['product']}")
    print(f"Top 3 proveedores:")
    for i, supplier in enumerate(simple_result['top_3'], 1):
        print(f"{i}. {supplier['name']} ({supplier['country']}): {supplier['recommendation']}")
        print(f"   Score: {supplier['score']} | Costo: ${supplier['total_cost']}")
else:
    print(f"Error: {simple_result['error']}")

print(f"\n" + "="*60)
print("SISTEMA CORREGIDO FUNCIONANDO CORRECTAMENTE!")
print("="*60)
print("DIFERENCIAS CON LA VERSIÓN ANTERIOR:")
print("1. Modelo SIN overfitting (accuracy ~0.6 en lugar de 1.0)")
print("2. NO usa country/quality_rating como features del modelo")
print("3. Combina ML score (40%) + factores externos (60%)")
print("4. Más realista y generalizable")
print("5. Listo para producción y microservicio")
print(f"\nUsar: recommend_suppliers('producto', 'urgencia', cantidad)")
print(f"Para UI: simple_search('producto', 'urgencia', cantidad)")

SISTEMA DE RECOMENDACIÓN DE PROVEEDORES - VERSIÓN CORREGIDA
Dewatering Solutions - Modelo sin Overfitting
CARGANDO MODELO CORREGIDO...
   Modelo corregido cargado correctamente
   Features del modelo: ['price_usd', 'delivery_days', 'payment_terms_days', 'shipping_included', 'express_available', 'order_urgency', 'quantity_needed', 'budget_available', 'product_type', 'incoterms', 'month', 'quarter']
   Proveedores disponibles: 6

INFORMACIÓN DE PROVEEDORES:
   AIR SLAID TECIDOS TÉCNICOS LTDA: Calidad 4.0/5.0, Brasil, 30 días
   Autonics Perú: Calidad 4.8/5.0, Perú, 5 días
   GWI GROUP LIMITED: Calidad 3.5/5.0, China, 28 días
   Ritherm: Calidad 4.5/5.0, Perú, 7 días
   TESTORI LATAM SpA: Calidad 5.0/5.0, Chile/Italia, 10 días
   Xiamen Borida Products Co., Ltd: Calidad 3.0/5.0, China, 22 días

EJEMPLOS DE USO DEL SISTEMA CORREGIDO

EJEMPLO 1: FILTROS DE PRENSA URGENTES
Producto: filter_press_cloth_set
Urgencia: High
Top 3 recomendaciones:
1. GWI GROUP LIMITED (China):
   Recomendación: R